In [1]:
import torch
import torchvision
from torch.utils import data
from torchvision import transforms
import torch.nn as nn
from torch.nn import functional as F
import collections
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import argparse
from PIL import Image
import re
import os
import math

In [2]:
def parse_arg():
    parser = argparse.ArgumentParser()
    parser.add_argument("--lr",type=float,default=1)
    parser.add_argument("--seed",type=int,default=5319)
    parser.add_argument("--batch_size",type=int,default=20)
    parser.add_argument("--num_steps",type=int,default=10)

    parser.add_argument("--num_hiddens",type=int,default=256)
    parser.add_argument("--num_epochs",type=int,default=500)
    parser.add_argument("--device",type=str,default="cuda")
    
    arg = parser.parse_args(args=[])
    return arg


args = parse_arg()


In [3]:
def read_time_machine():
    with open('./time_machine.txt', 'r') as f:
        lines = f.readlines()
    format_lines = [re.sub('[^A-Za-z]', ' ', line).strip().lower() for line in lines]
    lines = [line for line in format_lines if len(line)>0]
    return lines


def tokenizer(data,method='word'):
    ret = []
    if method=='word':
        for line in data:
            ret.extend(line.split())
        return ret
    elif method=='char':
        for line in data:
            ret.extend([i for i in line])
        return ret


def count_corpus(tokens):
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

time_machine_token = tokenizer(read_time_machine(),'word')
print(time_machine_token[:2000])
print(len(time_machine_token))

['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'time', 'machine', 'by', 'h', 'g', 'wells', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and', 'most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 're', 'use', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'www', 'gutenberg', 'org', 'if', 'you', 'are', 'not', 'located', 'in', 'the', 'united', 'states', 'you', 'will', 'have', 'to', 'check', 'the', 'laws', 'of', 'the', 'country', 'where', 'you', 'are', 'located', 'before', 'using', 'this', 'ebook', 'title', 'the', 'time', 'machine', 'author', 'h', 'g', 'wells', 'release', 'date', 'july', 'ebook', 'most', 'recently', 'updated', 'october', 'language', 'english', 'character', 'set', 'encoding', 'utf', 'star

In [4]:
class Vocab():
    def __init__(self,tokens,min_frequence=0,reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        counter = count_corpus(tokens)
        self.__token_freq = sorted(counter.items(),key=lambda x:x[1],reverse=True)
        self.index_to_token = ['<unk>'] + reserved_tokens
        self.unk = 0
        self.token_to_index = {token:idx for idx,token in enumerate(self.index_to_token)}
        for token,freq in self.token_freq:
            if freq <min_frequence:
                break
            if token not in self.index_to_token:
                self.index_to_token.append(token)
                self.token_to_index[token] = len(self.index_to_token) - 1
    
    def __len__(self):
        return len(self.index_to_token)
    
    def to_index(self,tokens):
        if not isinstance(tokens,(list,tuple)):
            return self.token_to_index.get(tokens,self.unk)
        return [self.to_index(token) for token in tokens]

    def to_token(self,indices):
        if not isinstance(indices,(tuple,list)):
            return self.index_to_token[indices]
        return [self.to_token(index) for index in indices]

    @property
    def token_freq(self):
        return self.__token_freq

time_machine_corpus = Vocab(time_machine_token)

In [5]:
# paired_token = [pair for pair in zip(time_machine_token[:-1],time_machine_token[1:])]
# paired_corpus = Vocab(paired_token)
# for i in range(10):
#     print(paired_corpus.token_freq[i])

# tri_token = [pair for pair in zip(time_machine_token[:-2],time_machine_token[1:-1],time_machine_token[2:])]
# tri_corpus = Vocab(tri_token)
# for i in range(10):
#     print(tri_corpus.token_freq[i])

In [6]:
# fig,axes = plt.subplots(2,2)
# x = [i for i in range(len(time_machine_corpus)-1)]
# y = [time_machine_corpus.token_freq[j][1] for j in x]
# print(y)
# print(time_machine_corpus.token_freq)
# axes[0][0].plot(x,x)
# axes[0][1].plot(x,y)
# axes[1][1].plot(x,y)
# axes[1][1].semilogx()
# axes[1][1].semilogy()
# plt.show()

In [7]:
def seq_data_iter_random(corpus, batch_size, num_steps):
    corpus = corpus[random.randint(0, num_steps - 1):]
    num_subseqs = (len(corpus)-1) // num_steps
    initial_indices = list(range(0, num_subseqs * num_steps, num_steps))
    random.shuffle(initial_indices)

    def data(pos):
        return corpus[pos: pos + num_steps]
    
    num_batches = num_subseqs // batch_size
    for i in range(0, batch_size * num_batches, batch_size):
        initial_indices_per_batch = initial_indices[i: i + batch_size]
        X = [data(j) for j in initial_indices_per_batch]
        Y = [data(j + 1) for j in initial_indices_per_batch]
        yield torch.tensor(X), torch.tensor(Y)


def seq_data_iter_sequential(corpus, batch_size, num_steps):
    offset = random.randint(0, num_steps - 1)
    num_tokens = ((len(corpus) - offset - 1) // batch_size) * batch_size
    Xs = torch.tensor(corpus[offset: offset + num_tokens])
    Ys = torch.tensor(corpus[offset + 1: offset + 1 + num_tokens])
    Xs, Ys = Xs.reshape(batch_size, -1), Ys.reshape(batch_size, -1)
    num_batches = Xs.shape[1] // num_steps
    for i in range(0, num_steps * num_batches, num_steps):
        X = Xs[:, i: i + num_steps]
        Y = Ys[:, i: i + num_steps]
        yield X, Y

In [8]:
class SeqDataLoader:
    def __init__(self, token ,batch_size, num_steps, use_random_iter):
        if use_random_iter:
            self.data_iter_fn = seq_data_iter_random
        else:
            self.data_iter_fn = seq_data_iter_sequential
        self.batch_size, self.num_steps = batch_size, num_steps
        self.token_index = time_machine_corpus.to_index(token)


    def __iter__(self):
        return self.data_iter_fn(self.token_index, self.batch_size, self.num_steps)
    
    def __len__(self):
        return len(self.token_index)
    
data_iter = SeqDataLoader(time_machine_token,batch_size=args.batch_size,num_steps=args.num_steps,use_random_iter=True)
for i,j in data_iter:
    print(time_machine_corpus.to_token(i.detach().numpy().tolist()),'\n',time_machine_corpus.to_token(j.detach().numpy().tolist()))
    print()
    print(time_machine_corpus.to_token(i.T.detach().numpy().tolist()),'\n',time_machine_corpus.to_token(j.T.detach().numpy().tolist()))
    print(i.shape,'\n',j.shape)
    break

[['good', 'for', 'he', 'looked', 'round', 'the', 'table', 'and', 'the', 'ghost'], ['like', 'children', 'they', 'would', 'soon', 'stop', 'examining', 'me', 'and', 'wander'], ['medical', 'man', 'seemed', 'absorbed', 'in', 'the', 'contemplation', 'of', 'our', 'host'], ['as', 'they', 'hurried', 'after', 'me', 'in', 'a', 'moment', 'i', 'was'], ['his', 'eye', 'fell', 'with', 'a', 'mute', 'inquiry', 'upon', 'the', 'withered'], ['author', 'h', 'g', 'wells', 'release', 'date', 'july', 'ebook', 'most', 'recently'], ['odour', 'of', 'camphor', 'was', 'unmistakable', 'in', 'the', 'universal', 'decay', 'this'], ['a', 'tangled', 'waste', 'of', 'beautiful', 'bushes', 'and', 'flowers', 'a', 'long'], ['being', 'on', 'easy', 'terms', 'in', 'the', 'house', 'i', 'went', 'up'], ['the', 'sky', 'was', 'no', 'longer', 'blue', 'north', 'eastward', 'it', 'was'], ['permission', 'for', 'the', 'use', 'of', 'the', 'work', 'and', 'the', 'project'], ['feet', 'running', 'and', 'stumbling', 'this', 'way', 'and', 'that',

In [9]:
def grad_clipping(net, theta):
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [10]:
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        if not self.rnn.bidirectional:
            self.num_directions = 1
            self.linear = nn.Linear(self.num_hiddens, self.vocab_size)
        else:
            self.num_directions = 2
            self.linear = nn.Linear(self.num_hiddens * 2, self.vocab_size)

    def forward(self, inputs, state):
        X = F.one_hot(inputs.T.long(), self.vocab_size)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state

    def begin_state(self, device, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
            # nn.GRU以张量作为隐状态
            return  torch.zeros((self.num_directions * self.rnn.num_layers,
                                 batch_size, self.num_hiddens),
                                device=device)
        else:
            # nn.LSTM以元组作为隐状态
            return (torch.zeros((
                self.num_directions * self.rnn.num_layers,
                batch_size, self.num_hiddens), device=device),
                    torch.zeros((
                self.num_directions * self.rnn.num_layers,
                batch_size, self.num_hiddens), device=device))
        
num_hiddens = args.num_hiddens
rnn_layer = nn.RNN(len(time_machine_corpus), num_hiddens)
rnn_net = RNNModel(rnn_layer=rnn_layer,vocab_size=len(time_machine_corpus))

In [11]:
def predict(prefix, num_preds, net, vocab, device):
    net.to(device)
    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab.to_index(prefix[0])]
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    for y in prefix[1:]:  # 预热期
        _, state = net(get_input(), state)
        outputs.append(vocab.to_index(y))
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ' '.join([vocab.to_token(i) for i in outputs])


def train_epoch(net, train_iter, loss, updater, device, use_random_iter):
    state =  None
    for X, Y in train_iter:
        if state is None or use_random_iter:
            state = net.begin_state(batch_size=X.shape[0], device=device)
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # state对于nn.GRU是个张量
                state.detach_()
            else:
                # state对于nn.LSTM或对于我们从零开始实现的模型是个张量
                for s in state:
                    s.detach_()
        y = Y.T.reshape(-1)
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, state)
        l = loss(y_hat, y.long()).mean()

        updater.zero_grad()
        l.backward()
        grad_clipping(net, 1)
        updater.step()
        loss_num = float(l.detach().cpu().numpy())
        train_iter.set_postfix_str(f'loss:{loss_num:.3f}')


def trainer(net, train_iter, vocab, lr, num_epochs, device, use_random_iter=False):
    loss = nn.CrossEntropyLoss()
    updater = torch.optim.SGD(net.parameters(), lr)
    predict_fn = lambda prefix: predict(prefix, 100, net, vocab, device)
    net.to(device)
    # 训练和预测
    for epoch in range(num_epochs):
        bar = tqdm(train_iter,total=len(train_iter)/(args.batch_size*args.num_steps),unit_scale=args.batch_size*args.num_steps)
        bar.set_description_str(f'[epoch:{epoch+1}/{num_epochs}]')
        train_epoch(net, bar, loss, updater, device, use_random_iter)
        if (epoch + 1) % 10 == 0:
            print(predict_fn(['traveller']))
    print(predict_fn(['traveller']))
    print(predict_fn(['traveller']))

In [12]:
predict('traveller',100,rnn_net,time_machine_corpus,args.device)

't <unk> a v e <unk> <unk> e <unk> merriment dried dried white slightness g weary white g consoled problems before arrears dried dried dried dried white white g suggestive problems before arrears dried dried dried dried white white g suggestive problems before arrears dried dried dried dried white white g suggestive problems before arrears dried dried dried dried white white g suggestive problems before arrears dried dried dried dried white white g suggestive problems before arrears dried dried dried dried white white g suggestive problems before arrears dried dried dried dried white white g suggestive problems before arrears dried dried dried dried white white g suggestive problems before'

In [13]:
trainer(rnn_net, data_iter, vocab=time_machine_corpus, lr=args.lr, num_epochs=args.num_epochs,device=args.device,use_random_iter=True)

[epoch:1/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 39124.08it/s, loss:6.586]
[epoch:2/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 67149.87it/s, loss:6.115]
[epoch:3/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 61268.16it/s, loss:6.220]
[epoch:4/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 63933.74it/s, loss:5.799]
[epoch:5/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 60340.56it/s, loss:6.007]
[epoch:6/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 67341.57it/s, loss:5.941]
[epoch:7/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 67658.07it/s, loss:5.544]
[epoch:8/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 67977.80it/s, loss:6.134]
[epoch:9/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 66592.26it/s, loss:5.747]
[epoch:10/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 64219.53it/s, loss:5.592]


traveller and the morlocks of the work and i had to the time machine and the foundation of the work and i had to the time machine was a little thing of the machine was in the time machine i had to see it was a little thing of the machine was in the time machine i had to see it was a little thing of the machine was in the time machine i had to see it was a little thing of the machine was in the time machine i had to see it was a little thing of the


[epoch:11/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 66166.56it/s, loss:5.464]
[epoch:12/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 65923.30it/s, loss:5.159]
[epoch:13/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 63819.95it/s, loss:5.548]
[epoch:14/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 63090.77it/s, loss:5.383]
[epoch:15/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 60903.50it/s, loss:5.299]
[epoch:16/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 60748.30it/s, loss:5.348]
[epoch:17/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 58056.49it/s, loss:5.450]
[epoch:18/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 58909.61it/s, loss:5.437]
[epoch:19/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 58529.07it/s, loss:5.169]
[epoch:20/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 59791.64it/s, loss:4.827]


traveller of the world i had seen to the sun and the white of the time machine and i was a small of the white and the other of the time traveller s i was a small of the world and the other of the time traveller and i was a small of the white and i saw the time traveller was a little thing i saw a little of a sudden of a creature i could not a mere of a sudden of a creature i could not a mere of a sudden of a creature i could not a


[epoch:21/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 60750.45it/s, loss:4.868]
[epoch:22/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 59693.55it/s, loss:4.931]
[epoch:23/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 60141.39it/s, loss:5.139]
[epoch:24/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 58575.57it/s, loss:4.999]
[epoch:25/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 59693.76it/s, loss:4.361]
[epoch:26/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 61161.96it/s, loss:4.724]
[epoch:27/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 57044.30it/s, loss:4.531]
[epoch:28/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 58959.69it/s, loss:4.373]
[epoch:29/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 61267.46it/s, loss:4.992]
[epoch:30/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 59792.73it/s, loss:4.734]


...

traveller s return so i came and the fa ade had an oriental look they came and as being a very strange and the creature though some time brightening and truly taking it will he hesitated his eye over the shoulder of his story he said what we thought of the time traveller any more than they not now it is to stick in the sunlight as if he could not the morlocks as i went on the machine you showed that it s too when i saw that i could sleep well that gallery and the red view of the


[epoch:481/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 58346.78it/s, loss:1.007]
[epoch:482/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 57369.64it/s, loss:0.876]
[epoch:483/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 60149.27it/s, loss:1.186]
[epoch:484/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 59504.29it/s, loss:1.034]
[epoch:485/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 60351.02it/s, loss:1.076]
[epoch:486/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 59602.76it/s, loss:0.993]
[epoch:487/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 60861.20it/s, loss:0.943]
[epoch:488/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 59190.14it/s, loss:1.040]
[epoch:489/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 60913.84it/s, loss:0.911]
[epoch:490/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 57645.41it/s, loss:0.975]


traveller s shoulder you don t believe it well from below the mystery if the earth s not trace this line of it in my pocket and now a white sphinx upon the further in the light the time traveller was not there were three circumstances in particular which made me when i saw me i had the hardest task in the darkness i had walked apparently this section me with it a peculiar a minute he put in my hand for a match and a white man of our own times and as its knows now that had a dim


[epoch:491/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 60758.89it/s, loss:0.847]
[epoch:492/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 60301.62it/s, loss:1.046]
[epoch:493/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 59454.25it/s, loss:1.086]
[epoch:494/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 59405.90it/s, loss:0.884]
[epoch:495/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 59552.29it/s, loss:1.004]
[epoch:496/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 60656.06it/s, loss:0.884]
[epoch:497/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 58919.98it/s, loss:1.040]
[epoch:498/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 58967.36it/s, loss:1.029]
[epoch:499/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 58346.72it/s, loss:1.022]
[epoch:500/500]: 100%|█████████▉| 36000/36019.0 [00:00<00:00, 58678.58it/s, loss:0.836]


traveller s face iv time travelling and this time travelling i had mastered the problem of the world mastered the whole secret of these delicious people possibly the checks they had devised for the increase of population had succeeded too well and these things to the pedestal it appeared to be broken in a corner of light among the eloi were before the old might be hidden in a corner and then stopped the fire came on and the increased grew upon the south westward towards the time machine and very vaguely there came a suggestion towards the solution of the
traveller s face iv time travelling and this time travelling i had mastered the problem of the world mastered the whole secret of these delicious people possibly the checks they had devised for the increase of population had succeeded too well and these things to the pedestal it appeared to be broken in a corner of light among the eloi were before the old might be hidden in a corner and then stopped the fire came on and the increased g